In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from os import listdir
sessions_df = pd.read_csv("data_toolkit/data/sessions.csv")
practices_df = pd.read_csv("data_toolkit/data/practices.csv")
ccgs_df = pd.read_csv("data_toolkit/data/ccgs.csv")

In [2]:
sessions_df.count()

id                      128618
practice_id             128618
posted_datetime         128618
start_datetime          128618
end_datetime            128618
hourly_rate             128618
original_hourly_rate     87316
status                  128618
locum_id                 47732
dtype: int64

In [3]:
import datetime

length_seconds = lambda row: (row['end_datetime'] - row['start_datetime']).total_seconds()
one_hour = 60 * 60
is_short = lambda row: row['length_seconds'] <= 2 * one_hour
is_medium = lambda row: 2 * one_hour < row['length_seconds'] <= 4 * one_hour
is_long = lambda row: 4 * one_hour < row['length_seconds']


# These methods may exclude sessions that are very very long, but those
# appear to be outliers. Keeping it simple. 
def at_night(row): 
    return row['start_datetime'].time() >= datetime.time(18) or \
           row['end_datetime'].time() <= datetime.time(8)
   
   
def at_morning(row):
    start_in_morning = datetime.time(11) >= row['start_datetime'].time() >= datetime.time(6)
    end_in_morning = datetime.time(8) >= row['end_datetime'].time() >= datetime.time(12)
    if start_in_morning or end_in_morning:
        return True
    else:
        return False
    

def at_afternoon(row):
    start_in_aft = datetime.time(16) >= row['start_datetime'].time() > datetime.time(12)
    end_in_aft = datetime.time(20) >= row['end_datetime'].time() >= datetime.time(15)
    if start_in_aft or end_in_aft:
        return True
    else:
        return False
    
def is_on_day_of_week_partial(day_of_week):
    def is_on_day_of_week(row):
        if row['start_datetime'].weekday() == day_of_week:
            return True
        else:
            return False
    return is_on_day_of_week

In [4]:
# Transform data
# Add new features that come from the session time
sessions_df['start_datetime'] = pd.to_datetime(sessions_df['start_datetime'])
sessions_df['end_datetime'] = pd.to_datetime(sessions_df['end_datetime'])

sessions_df['length_seconds'] = sessions_df[['start_datetime', 'end_datetime']].apply(
    length_seconds, axis=1
)
sessions_df['is_short'] = sessions_df[['length_seconds']].apply(
    is_short, axis=1
)
sessions_df['is_medium'] = sessions_df[['length_seconds']].apply(
    is_medium, axis=1
)
sessions_df['is_long'] = sessions_df[['length_seconds']].apply(
    is_long, axis=1
)

sessions_df['at_night'] = sessions_df[['start_datetime', 'end_datetime']].apply(
    at_night, axis=1
)
sessions_df['at_afternoon'] = sessions_df[['start_datetime', 'end_datetime']].apply(
    at_afternoon, axis=1
)
sessions_df['at_morning'] = sessions_df[['start_datetime', 'end_datetime']].apply(
    at_morning, axis=1
)

sessions_df['monday'] = sessions_df[['start_datetime']].apply(
    is_on_day_of_week_partial(0), axis=1
)
sessions_df['tuesday'] = sessions_df[['start_datetime']].apply(
    is_on_day_of_week_partial(1), axis=1
)
sessions_df['wednesday'] = sessions_df[['start_datetime']].apply(
    is_on_day_of_week_partial(2), axis=1
)
sessions_df['thursday'] = sessions_df[['start_datetime']].apply(
    is_on_day_of_week_partial(3), axis=1
)
sessions_df['friday'] = sessions_df[['start_datetime']].apply(
    is_on_day_of_week_partial(4), axis=1
)
sessions_df['saturday'] = sessions_df[['start_datetime']].apply(
    is_on_day_of_week_partial(5), axis=1
)
sessions_df['sunday'] = sessions_df[['start_datetime']].apply(
    is_on_day_of_week_partial(6), axis=1
)

In [75]:
# Clean data
# We care about Completed / Filled / Expired sessions
clean_df = sessions_df[
    (sessions_df.status == 'completed') |
    (sessions_df.status == 'expired') |
    (sessions_df.status == 'filled')
]
clean_df['filled'] = clean_df[['status']].apply(
    lambda row: row['status'] in ['completed', 'filled'], axis=1
)

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [76]:
display(clean_df.describe())

,id,practice_id,hourly_rate,original_hourly_rate,locum_id,length_seconds
count,7.187300e+04,7.187300e+04,71873.000000,56544.000000,4.391500e+04,71873.000000
mean,3.686400e+08,2.139361e+06,81.944086,82.027766,1.522447e+07,17365.769379
std,1.033498e+08,1.279175e+06,7.199402,6.891379,5.001106e+07,8211.822889
min,4.016688e+07,3.011200e+04,60.000000,1.850000,2.011000e+04,7200.000000
25%,4.011212e+08,3.019170e+05,80.000000,80.000000,2.012139e+06,10800.000000
50%,4.011550e+08,3.011153e+06,80.000000,80.000000,2.013710e+06,14400.000000
75%,4.011850e+08,3.011767e+06,85.000000,85.000000,2.015509e+06,21600.000000
max,4.012203e+08,3.012808e+06,150.000000,150.000000,2.017111e+08,50400.000000


In [78]:
# merge ccg ids into the dataframe
clean_df = pd.merge(clean_df, practices_df, left_on='practice_id', right_on='id')

In [79]:
clean_df.loc[clean_df['locum_id'].isnull(), 'locum_id'] = 0

In [127]:
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import f1_score, accuracy_score

import datetime

def predict_and_score(regressor, fields_to_learn_on):
    start = datetime.datetime.now()
    X_train, X_test, y_train, y_test = train_test_split(
        clean_df[fields_to_learn_on],
        clean_df['filled'],
        train_size=int(sessions_df.count()[0] * 0.25), 
        random_state=1
    )
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    y_pred_bool = [1 if y >= 0.5 else 0 for y in y_pred]
    f1 = f1_score(y_test.values, y_pred_bool)
    accuracy_pct = accuracy_score(y_test.values, y_pred_bool)
    print("time to run: {}".format((datetime.datetime.now() - start).total_seconds()))
    return f1, accuracy_pct

# test/train split, then see the performance for different predictors

# train on all the data, then make a graph using the predictor:
# * a line each combination of bools
# * y axis is fill rate likelihood
# * x axis is hourly rate

In [115]:
day_of_week_fields = [
    'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday',
]
time_of_day_fields = [
    'at_night', 'at_afternoon', 'at_morning',
]
length_fields = [
    'is_short', 'is_medium', 'is_long',
]
time_fields = day_of_week_fields + time_of_day_fields + length_fields 
all_fields_to_learn_on = [
    'hourly_rate',
    'ccg_id', 'practice_id', 'locum_id'
] + time_fields

results = []

result = "decision tree on all fields: f1: {} accuracy: {}".format(
    *predict_and_score(
        DecisionTreeRegressor(random_state=1), all_fields_to_learn_on
))
print(result)
results.append(result)

decision tree on all fields: f1: 0.9997534719368888 accuracy: 0.9996978775900702


In [117]:
result = "decision tree on non-locum fields: f1: {} accuracy: {}".format(
    *predict_and_score(
        DecisionTreeRegressor(random_state=1), 
        ['hourly_rate', 'ccg_id', 'practice_id'] + time_fields
))
print(result)
results.append(result)

decision tree on non-locum fields: f1: 0.8400190642624513 accuracy: 0.7971751554671568


In [118]:
result = "decision tree on non-locum/practice id fields: f1: {} accuracy: {}".format(
    *predict_and_score(
        DecisionTreeRegressor(random_state=1), 
        ['hourly_rate', 'ccg_id'] + time_fields
))
print(result)
results.append(result)

decision tree on non-locum/practice id fields: f1: 0.8330298223971491 accuracy: 0.7853168508774139


In [119]:
result = "decision tree on non-locum/practice/ccg fields: f1: {} accuracy: {}".format(
    *predict_and_score(
        DecisionTreeRegressor(random_state=1), 
        ['hourly_rate',] + time_fields
))
print(result)
results.append(result)

decision tree on non-locum/practice/ccg fields: f1: 0.8023506881207637 accuracy: 0.7349631158891211


In [120]:
result = "decision tree on non-locum/hourly rate fields: f1: {} accuracy: {}".format(
    *predict_and_score(
        DecisionTreeRegressor(random_state=1), 
        ['practice_id', 'ccg_id'] + time_fields
))
print(result)
results.append(result)

decision tree on non-locum/hourly rate fields: f1: 0.8411863237757414 accuracy: 0.7970996248646743


In [121]:
result = "decision tree on hourly rate + ccg + practice id: f1: {} accuracy: {}".format(
    *predict_and_score(
        DecisionTreeRegressor(random_state=1), 
        ['hourly_rate', 'ccg_id', 'practice_id']
))
print(result)
results.append(result)

decision tree on hourly rate + ccg + practice id: f1: 0.8531287911399835 accuracy: 0.8110224325889372


In [122]:
result = "decision tree on hourly rate + ccg_id fields: f1: {} accuracy: {}".format(
    *predict_and_score(
        DecisionTreeRegressor(random_state=1), 
        ['hourly_rate', 'ccg_id']
))
print(result)
results.append(result)

decision tree on hourly rate + ccg_id fields: f1: 0.8273270642291144 accuracy: 0.7767063621944158


In [123]:
for result in results:
    print(result)

decision tree on all fields: f1: 0.9997534719368888 accuracy: 0.9996978775900702
decision tree on non-locum fields: f1: 0.8400190642624513 accuracy: 0.7971751554671568
decision tree on non-locum fields: f1: 0.8400190642624513 accuracy: 0.7971751554671568
decision tree on non-locum/practice id fields: f1: 0.8330298223971491 accuracy: 0.7853168508774139
decision tree on non-locum/practice/ccg fields: f1: 0.8023506881207637 accuracy: 0.7349631158891211
decision tree on non-locum/hourly rate fields: f1: 0.8411863237757414 accuracy: 0.7970996248646743
decision tree on hourly rate + ccg + practice id: f1: 0.8531287911399835 accuracy: 0.8110224325889372
decision tree on hourly rate + ccg_id fields: f1: 0.8273270642291144 accuracy: 0.7767063621944158


In [128]:
from sklearn.svm import SVR
result = "SVR on time, hourly rate + ccg_id + practice_id fields: f1: {} accuracy: {}".format(
    *predict_and_score(
        SVR(), 
        time_fields + ['hourly_rate', 'ccg_id', 'practice_id']
))
print(result)
results.append(result)

time to run: 142.66006
SVR on hourly rate + ccg_id fields: f1: 0.856995578817978 accuracy: 0.8183992547647222


In [130]:
from sklearn.svm import SVR
result = "SVR on time, practice_id + ccg_id fields: f1: {} accuracy: {}".format(
    *predict_and_score(
        SVR(), 
        time_fields + ['ccg_id', 'practice_id']
))
print(result)
results.append(result)

time to run: 130.720498
SVR on time, practice_id + ccg_id fields: f1: 0.8534508308470491 accuracy: 0.8132631737959163


In [131]:
0.856995578817978 - 0.8534508308470491
# the predictive power of adding hourly rate
# hourly rate could be explained in practice_id for its predictive power

0.003544747970928963